[View in Colaboratory](https://colab.research.google.com/github/manksh/K-nearest-neighbors/blob/master/assign2_part2.ipynb)

Assignment 2 - Part 2 

Author: Manksh Gupta (mg3835)

# Imports

In [0]:
import tensorflow as tf
tf.enable_eager_execution()
tfe = tf.contrib.eager
import numpy as np
import matplotlib.pyplot as plt
import warnings

In [0]:
plt.rcParams["figure.figsize"] = (10,7)
warnings.filterwarnings("ignore")
import os

# Data

In [0]:
mnist = tf.keras.datasets.mnist

# Dataset will be cached locally after you run this code
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Normalize pixel values to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# These types are required for the operation we use to compute
# loss. Omit, and you shall receive a cryptic error message.
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

In [0]:
slices = np.random.random(10000)<0.8
x_validate = x_test[~slices]
y_validate = y_test[~slices]
x_test = x_test[slices]
y_test = y_test[slices]

In [0]:
buffer_size = 5000
batch_size = 100

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(buffer_size)
train_dataset = train_dataset.batch(batch_size)

# Models

### Deep Model

In [0]:
class Model_deep(tf.keras.Model):
      def __init__(self,a, active):
        self.a = a
        self.active = active
        super(Model_deep, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense_1 = tf.keras.layers.Dense(128, activation=active, kernel_initializer=a)
        self.dropout_1 = tf.keras.layers.Dropout(0.2)
        self.dense_2 = tf.keras.layers.Dense(64, activation=active, kernel_initializer=a)
        self.dropout_2 = tf.keras.layers.Dropout(0.2)
        self.dense_3 = tf.keras.layers.Dense(64, activation=active, kernel_initializer=a)
        self.dropout_3 = tf.keras.layers.Dropout(0.2)
        self.dense_4 = tf.keras.layers.Dense(10)
    # FIX ME
    # add some layers to your model

      def call(self, x):
        x = self.flatten(x)
        x = self.dense_1(x)
        x = self.dropout_1(x)
        x = self.dense_2(x)
        x = self.dropout_2(x)
        x = self.dense_3(x)
        x = self.dropout_3(x)
        x = self.dense_4(x)
        
    # FIX ME
    # use your layers (don't forget to add activation functions here as well
    # if you haven't specified them in your layer definintions)
    
    
        return x # be sure to return logits, not softmax output

In [0]:
def train_deep(model_deep, images, labels, optim):
  # You should not need to modify this function
  with tf.GradientTape() as tape:
    logits = model_deep(images)
    loss_value = loss(logits, labels)  
    grads = tape.gradient(loss_value, model_deep.variables)
    optim.apply_gradients(zip(grads, model_deep.variables))

  return loss_value

### Linear Model

In [0]:
class Model_linear(tf.keras.Model):
      def __init__(self, a):
        self.a = a
        super(Model_linear, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense_4 = tf.keras.layers.Dense(10, kernel_initializer=self.a)
    # FIX ME
    # add some layers to your model

      def call(self, x):
        x = self.flatten(x)
        x = self.dense_4(x)
        
    # FIX ME
    # use your layers (don't forget to add activation functions here as well
    # if you haven't specified them in your layer definintions)
    
    
        return x # be sure to return logits, not softmax output

In [0]:
def train_linear(model_linear, images, labels, optim):
  # You should not need to modify this function
  with tf.GradientTape() as tape:
    logits = model_linear(images)
    loss_value = loss(logits, labels)  
  grads = tape.gradient(loss_value, model_linear.variables)
  optim.apply_gradients(zip(grads, model_linear.variables))
  return loss_value

### Common Functions

In [0]:
def loss(logits, labels):
  # FIX ME
  # You will need to modify this function, of course.
  # Best bet, use tf.nn.sparse_softmax_cross_entropy_with_logits
  # though if you're interested, you can write your own.
    
  return tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = labels))

In [0]:
def compute_accuracy(logits, labels):
  # You shoud not need to modify this function
  predictions = tf.argmax(logits, axis=1)
  batch_size = int(logits.shape[0])
  return tf.reduce_sum(tf.cast(tf.equal(predictions, labels), dtype=tf.float32)) / batch_size

In [0]:
# The first time you run the below block it will crash
# with an error 'ValueError: No variables provided.''
# This is because the call method of your model
# is not using any trainable variables.
# (As written, the model just flattens the images.)

def run_model(mod, optimizer, epoch):
    optimizer = optimizer
    model = mod
    epochs = epoch
    step_counter = 0
    train_accu = []
    validation_accu = []
    train_loss = []

    for epoch_n in range(epochs):
      for (batch, (images, labels)) in enumerate(train_dataset):
        loss_value = train_linear(model, images, labels, optim = optimizer)
        step_counter +=1
        train_loss.append(loss_value)
        
        


      train_accuracy = compute_accuracy(model(x_train), y_train)
      validation_accuracy = compute_accuracy(model(x_validate), y_validate)


      train_accu.append(train_accuracy)
      validation_accu.append(validation_accuracy)
        

    test_accu = compute_accuracy(model(x_test), y_test)
    
    return train_accu, validation_accu, test_accu, train_loss

# Linear Model

Here, I am training a linear model and visualizing the loss.

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'loss')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.title('Visualizing Loss')
plt.legend()

In [0]:
"""# create a log directory
logdir = os.mkdir("logs")

# run an experiment and write logs
writer = tf.contrib.summary.create_file_writer(logdir="logs/exp_1", flush_millis=1000)
with writer.as_default():
    with tf.contrib.summary.always_record_summaries():
        for i,j in enumerate(list(np.array(train_loss))):
            loss = j
            tf.contrib.summary.scalar("loss", loss, step=i)
writer.close()

!tensorboard --logdir=logs
# open your browser http://localhost:6006/"""

In [0]:
plt.plot(train_accu, label = 'train_accuracy')
plt.plot(validation_accu, label = 'validation_accuracy ')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.title('Visualizing Accuracy')
plt.legend()
plt.ylim(0.5,1)

Now, I implement a linear model to make predictions and I will try different learning rates, different optimizers,  different weight initializations. Since this model has only 1 layer(output layer), I do not try different activations and stick to softmax.

### Different Optimizers

I will try the following optimizers:

1) Adam Optimizer

2) Vanilla Gradient Descent

3) AdaGrad Optimizer

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu1, validation_accu1, test_accu1, train_loss1 = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu2, validation_accu2, test_accu2, train_loss2 = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.AdagradOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'Adam')
plt.plot(train_loss1, label = 'Vanilla Gradient Descent')
plt.plot(train_loss2, label = 'AdaGrad')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()

In [0]:
plt.plot(train_accu, label = 'train_accuracy - Adam')
plt.plot(validation_accu, label = 'validation_accuracy - Adam')
plt.plot(train_accu1, label = 'train_accuracy - Vanilla Gradient Descent')
plt.plot(validation_accu1, label = 'validation_accuracy - Vanilla Gradient Descent')
plt.plot(train_accu2, label = 'train_accuracy - AdaGrad')
plt.plot(validation_accu2, label = 'validation_accuracy - AdaGrad')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()
plt.ylim(0.5,1)

### Different Learning Rates

I will now try different learning rates. In order to do this, I will fix Adam as the optimizer(as it worked the best) and try different learning rates.

1) 1 ( High learning rate)

2) 0.001 (Reasonable learning rate)

3) 0.0000001 (Low learning rate)

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 1), epoch = 15)

In [0]:
train_accu1, validation_accu1, test_accu1, train_loss1 = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu2, validation_accu2, test_accu2, train_loss2 = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.0000001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'learning_rate = 1')
plt.plot(train_loss1, label = 'learning_rate = 0.001')
plt.plot(train_loss2, label = 'learning_rate = 0.0000001')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()

In [0]:
plt.plot(train_accu, label = 'train_accuracy - learning_rate = 1')
plt.plot(validation_accu, label = 'validation_accuracy - learning_rate = 1')
plt.plot(train_accu1, label = 'train_accuracy - learning_rate = 0.001')
plt.plot(validation_accu1, label = 'validation_accuracy - learning_rate = 0.001')
plt.plot(train_accu2, label = 'train_accuracy - learning_rate = 0.0000001')
plt.plot(validation_accu2, label = 'validation_accuracy - learning_rate = 0.0000001')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()
plt.ylim(0.5,1)

For the linear model, significant difference is seen when changing the learning rate for the linear model. Learning rate of 0.001 seems to be the most optimal looking at the loss values.

### Different Weight Initializations

I will now try different weight initialization stratergys for the model. I will try:

1) Default

2) Random Uniform

3) Random Normal

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_linear('glorot_uniform_initializer'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu1, validation_accu1, test_accu1, train_loss1 = run_model(mod = Model_linear('RandomUniform'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu2, validation_accu2, test_accu2, train_loss2 = run_model(mod = Model_linear('RandomNormal'), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'Initializer = Default')
plt.plot(train_loss1, label = 'Initializer = RandomUniform')
plt.plot(train_loss2, label = 'Initializer - RandomNormal')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()

In [0]:
plt.plot(train_accu, label = 'train_accuracy - Initializer = Default')
plt.plot(validation_accu, label = 'validation_accuracy - Initializer = Default')
plt.plot(train_accu1, label = 'train_accuracy - Initializer = RandomUniform')
plt.plot(validation_accu1, label = 'validation_accuracy - Initializer = RandomUniform')
plt.plot(train_accu2, label = 'train_accuracy - Initializer - RandomNormal')
plt.plot(validation_accu2, label = 'validation_accuracy - Initializer - RandomNormal')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()
plt.ylim(0.5,1)

We see that the best one is the default one, the other ones take longer to converge.

# Deep Model

Here, I am training a Deep model and visualizing the loss.

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'Loss')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.title('Visualizing Loss')
plt.legend()

In [0]:
"""# create a log directory
logdir = os.mkdir("logs")

# run an experiment and write logs
writer = tf.contrib.summary.create_file_writer(logdir="logs/exp_1", flush_millis=1000)
with writer.as_default():
    with tf.contrib.summary.always_record_summaries():
        for i,j in enumerate(list(np.array(train_loss))):
            loss = j
            tf.contrib.summary.scalar("loss", loss, step=i)
writer.close()

!tensorboard --logdir=logs
# open your browser http://localhost:6006/"""

In [0]:
plt.plot(train_accu, label = 'train_accuracy')
plt.plot(validation_accu, label = 'validation_accuracy ')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.title('Visualizing Accuracy')
plt.legend()
plt.ylim(0.5,1)

Now, I implement the Deep model to make predictions and I will try different learning rates, different optimizers,  different weight initializations and different activations.

### Different Optimizers

I will try the following optimizers:

1) Adam Optimizer

2) AdaDelta

2) Vanilla Dradient Descent

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu1, validation_accu1, test_accu1, train_loss1 = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu2, validation_accu2, test_accu2, train_loss2 = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdagradOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'Adam')
plt.plot(train_loss1, label = 'Vanilla Gradient Descent')
plt.plot(train_loss2, label = 'AdaGrad')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()

In [0]:
plt.plot(train_accu, label = 'train_accuracy - Adam')
plt.plot(validation_accu, label = 'validation_accuracy - Adam')
plt.plot(train_accu1, label = 'train_accuracy - Vanilla Gradient Descent')
plt.plot(validation_accu1, label = 'validation_accuracy - Vanilla Gradient Descent')
plt.plot(train_accu2, label = 'train_accuracy - AdaGrad')
plt.plot(validation_accu2, label = 'validation_accuracy - AdaGrad')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()
plt.ylim(0.5,1)

### Different learning rates

I will now try different learning rates. In order to do this, I will fix Adam as the optimizer(as it performed the best) and try different learning rates.

1) 1 ( High learning rate)

2) 0.001 (Reasonable learning rate)

3) 0.0000001 (Low learning rate)

In [0]:

train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 1), epoch = 15)

In [0]:
train_accu1,validation_accu1, test_accu1, train_loss1 = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu2, validation_accu2, test_accu2, train_loss2 = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.0000001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'learning_rate = 1')
plt.plot(train_loss1, label = 'learning_rate = 0.001')
plt.plot(train_loss2, label = 'learning_rate = 0.0000001')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.ylim(0,5)
plt.legend()

In [0]:
plt.plot(train_accu, label = 'train_accuracy - learning_rate = 1')
plt.plot(validation_accu, label = 'validation_accuracy - learning_rate = 1')
plt.plot(train_accu1, label = 'train_accuracy - learning_rate = 0.001')
plt.plot(validation_accu1, label = 'validation_accuracy - learning_rate = 0.001')
plt.plot(train_accu2, label = 'train_accuracy - learning_rate = 0.0000001')
plt.plot(validation_accu2, label = 'validation_accuracy - learning_rate = 0.0000001')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()
plt.ylim(0,1)

### Different Weight Initializations

I will now try different weight initialization stratergys for the model. I will try:

1) Default

2) Random Uniform

3) Random Normal

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu1,validation_accu1, test_accu1, train_loss1 = run_model(mod = Model_deep('RandomUniform', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu2, validation_accu2, test_accu2, train_loss2 = run_model(mod = Model_deep('RandomNormal', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'Default')
plt.plot(train_loss1, label = 'RandomUniform')
plt.plot(train_loss2, label = 'RandomNormal')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()

In [0]:
plt.plot(train_accu, label = 'train_accuracy - Default')
plt.plot(validation_accu, label = 'validation_accuracy - Default')
plt.plot(train_accu1, label = 'train_accuracy - RandomUniform')
plt.plot(validation_accu1, label = 'validation_accuracy - RandomUniform')
plt.plot(train_accu2, label = 'train_accuracy - RandomNormal')
plt.plot(validation_accu2, label = 'validation_accuracy - RandomNormal')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()
plt.ylim(0.5,1)

### Different Activations

I will now try different weight initialization stratergys for the model. I will try:

1) relu

2) sigmoid

3) tanh

In [0]:
train_accu, validation_accu, test_accu, train_loss = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.relu), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu1, validation_accu1, test_accu1, train_loss1 = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.sigmoid), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
train_accu2, validation_accu2, test_accu2, train_loss2 = run_model(mod = Model_deep('glorot_uniform_initializer', active = tf.nn.tanh), 
                                                   optimizer = tf.train.AdamOptimizer(learning_rate = 0.001), epoch = 15)

In [0]:
plt.plot(train_loss, label = 'relu')
plt.plot(train_loss1, label = 'sigmoid')
plt.plot(train_loss2, label = 'tanh')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()

In [0]:
plt.plot(train_accu, label = 'train_accuracy - relu')
plt.plot(validation_accu, label = 'validation_accuracy - relu')
plt.plot(train_accu1, label = 'train_accuracy - sigmoid')
plt.plot(validation_accu1, label = 'validation_accuracy - sigmoid')
plt.plot(train_accu2, label = 'train_accuracy - tanh')
plt.plot(validation_accu2, label = 'validation_accuracy - tanh')
plt.axes().yaxis.grid(True)
plt.axes().xaxis.grid(True)
plt.legend()
plt.ylim(0.5,1)